# Data Management Sample
This sample shows you how to connect and use Azure Quantum notebooks with external datasources such as Azure Blob Storage. First, connect to your Azure Quantum Workspace:

In [ ]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "",
    resource_group = "",
    name = "",
    location = ""
)

## Work with data in Azure Blob Storage
This section will show you how to work with JSON and other file types in Blob Storage.

Your [Azure Quantum Workspace](https://docs.microsoft.com/azure/quantum/how-to-create-workspace?tabs=tabid-quick) has a storage account attached to it by default - this is normally used for management of job data but can also be used to store your data.

You can find details for this storage account by navigating to your Azure Quantum Workspace in the portal - it is shown in the 'Essentials' section at the top - if you click the name of the storage account it will take you to view it in the portal.

You can connect to this storage account directly through the Azure Portal or by using the free [Azure Storage Explorer desktop app](https://azure.microsoft.com/products/storage/storage-explorer/#overview). Both of these methods will allow you to view, upload, download and delete files and containers in your storage account.

For further information on the Azure Blob Storage service, please refer to the [docs page](https://docs.microsoft.com/azure/storage/blobs/storage-blobs-introduction).

### Connect to Blob Storage

The following code snippet shows how to create a connection to the storage account attached to your Azure Quantum workspace, and work with data stored therein. This snippet works specifically with a container named "data", which will not exist by default - the code below will create the container for you if it does not already exist.

In [ ]:
import json
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import ContainerClient


# Fetch URI for the Storage account attached to your Azure Quantum Workspace
# If the blob container 'data' does not exist already, it will be created
storage_uri = workspace.get_container_uri(container_name="data")

# Use this container client to interact with the "data" storage account container
container_client = ContainerClient.from_container_url(container_url=storage_uri)

### Upload JSON data to Blob Storage

The following snippet shows you how to upload JSON data to the newly-created "data" container within your storage account.

In [ ]:
# Define JSON object for upload
json_data_for_upload = {
    "data": {
        "type": "test",
        "value": 123
    }
}

# Create blob client for the new file
blob_client = container_client.get_blob_client(blob="file.json")

# Upload JSON data
# Enable/disable overwrite to specify behaviour when a blob already exists with that name
try:
    blob_client.upload_blob(json.dumps(json_data_for_upload), overwrite=True)
    print("Upload succeeded!")
except Exception as e:
    print("Upload failed! Reason: {e}")

### Download JSON data from Blob Storage

The following snippet shows you how to fetch data from a JSON file stored in the "data" container of the Storage Account attached to your Azure Quantum Workspace. For this to work, the file must previously have been uploaded to the blob storage container. For simplicity, this snippet makes use of the file you uploaded in the previous section.

In [ ]:
# Create blob client for your file
blob_client = container_client.get_blob_client(blob="file.json")

# Download file from blob storage & parse JSON into input_data object
download_stream = blob_client.download_blob()
input_data = json.loads(download_stream.readall())

# Display file contents
print(json.dumps(input_data, indent=2))

### Upload text/other filetypes to Blob Storage

In [ ]:
# Define text for upload
text_data_for_upload = "Some text to upload to a file. File could be .csv, .txt or any other format you choose."

# Create blob client for the new file (make sure to choose the appropriate extension e.g. .txt or .csv)
blob_client = container_client.get_blob_client(blob="file.txt")

# Upload data
# Enable/disable overwrite to specify behaviour when a blob already exists with that name
try:
    blob_client.upload_blob(text_data_for_upload, overwrite=True)
    print("Upload succeeded!")
except Exception as e:
    print("Upload failed! Reason: {e}")

### Download text/other filetypes from Blob Storage

In [ ]:
# Create blob client for your file
blob_client = container_client.get_blob_client(blob="file.txt")

# Download file from blob storage & assign to input_data bytes object
download_stream = blob_client.download_blob()
input_data = download_stream.readall().decode("utf-8") 

print(input_data)

## Work with data from an external URL
The following code snippet shows you how to load data from a URL (with no authentication) and save the data to a file in Blob Storage.

### Download data from URL

In [ ]:
# Import requests module
import requests

# Paste the URL for your file here
# This example loads the Broombridge 2.0 schema file from the Microsoft Quantum samples repo
url = "https://raw.githubusercontent.com/microsoft/Quantum/main/Chemistry/Schema/broombridge-0.2.schema.json"

# Get data from URL
r = requests.get(url)

# Work with the data - e.g. could upload to Blob Storage after processing
# See https://www.w3schools.com/PYTHON/ref_requests_response.asp for further properties of the requests.Response object which you can use for processing
data_bytes = r.content
data_json = r.json()
data_text = r.text

print(f"Title: {data_json['title']}")

## Work with files in local storage (temporary storage)
You can use the following code snippets to write data to local storage. Local storage is treated as a temporary storage location as it is hard to locate local files afterwards and the storage may not persist across sessions. It is therefore recommended that you only use this for temporary file operations and use the Blob Storage access methods provided in this sample to more permanently store your data.

In [ ]:
# Define JSON data to save
json_data = {
    "data": {
        "type": "test",
        "value": 123
    }
}

# Write JSON to local storage
with open("file.json", 'w') as f:
    f.write(json.dumps(json_data))

# Define text data to save
text_data = "Some text! Hello :)"

# Write text to local storage
with open("file.txt", 'w') as f:
    f.write(text_data)

# Read JSON from local storage
with open("file.json") as fp:
    input_data = json.load(fp)

print(f"JSON data: \n{json.dumps(input_data, indent=2)}")

# Read text file from local storage
with open("file.txt") as fp:
    input_data = fp.read()
    
print(f"\nText data:\n{input_data}")

## End-to-end example

The sample below shows how to load a spin-orbital Hamiltonian from a file containing orbital integrals. Features of the Hamiltonian are then computed and the results are saved to a .csv file in Blob Storage. The input data is loaded from a publicly-accessible GitHub URL.

The code for this example is taken directly from the [chemistry section of the samples repo](https://github.com/microsoft/Quantum/tree/main/samples/chemistry). This specific example computes the features for Ozone:

> Ozone is formed from dioxygen by the action of ultraviolet light (UV) and electrical discharges within the Earth's atmosphere. It is present in very low concentrations throughout the latter, with its highest concentration high in the ozone layer of the stratosphere, which absorbs most of the Sun's ultraviolet (UV) radiation. Quantum mechanical excited-state studies, including localization of avoided crossings and conical intersection play a critical role in understanding its role in Earth atmosphere.

### Compute Hamiltonian features for the Ozone molecule
You can find further orbital integral data in the samples repo [here](https://github.com/microsoft/Quantum/tree/main/samples/chemistry/IntegralData/YAML).

To load data from a GitHub URL using the `requests` module, you need to use the 'raw' content URL (should have `raw.githubusercontent.com` as the domain). 

You can find this URL by navigating to the file you would like to use and selecting 'Raw' from the panel at the top of the document view. This should reload the page - use the URL for the page that just loaded.

In [ ]:
import requests, json
import numpy as np
from numpy import linalg as LA
from qsharp.chemistry import load_broombridge, load_fermion_hamiltonian, IndexConvention
from azure.storage.blob import BlobServiceClient
from azure.quantum import Workspace

# Fetch data from URL
# See note above for instructions on how to test with other files
r = requests.get("https://raw.githubusercontent.com/microsoft/Quantum/main/samples/chemistry/IntegralData/YAML/O3_ccpvtz/o3_13_6_6_90deg_ccvtz.yaml")

ozone_filename = "o3_13_6_6_90deg_ccvtz.yaml"

# Write file to local storage (temp storage)
with open(ozone_filename, 'wb') as f:
    f.write(r.content) 

print(f"Processing the following file: {ozone_filename}\n")

# Load Broombridge schema for Ozone
broombridge = load_broombridge(ozone_filename)

# Load Hamiltonian data
general_hamiltonian = broombridge.problem_description[0].load_fermion_hamiltonian(index_convention=IndexConvention.UpDown)

# Calculate one-norms and save results to output string
output = "Type,one-norm\n"
print("Computing One-norms:")
for term, matrix in general_hamiltonian.terms:
    one_norm = LA.norm(np.asarray([v for k, v in matrix], dtype=np.float32), ord=1)
    output += f"{term},{one_norm}\n"
    print(f"\tOne-norm for term type {term}: {one_norm}")

print()

# Fetch URI for the Storage account attached to your Azure Quantum Workspace
# If the blob container 'data' does not exist already, it will be created
storage_uri = workspace.get_container_uri(container_name="data")

# Use this container client to interact with the storage account container
container_client = ContainerClient.from_container_url(container_url=storage_uri)

# Create blob client for the new .csv output file
blob_client = container_client.get_blob_client(blob="ozone_results.csv")

# Upload data to Blob Storage
# Enable/disable overwrite to specify behaviour when a blob already exists with that name
try:
    blob_client.upload_blob(output, overwrite=True)
    print("Results saved to blob storage!")
except Exception as e:
    print("Upload failed! Reason: {e}")

If you ran all cells in this sample notebook, you should now be able to see a new container called `data` in the Azure Blob Storage account associated with your Azure Quantum Workspace. It should have three files in it:

- file.json
- file.txt
- ozone_results.csv